### 【 Model save & load 】
- 2가지 형태 저장
    * 형태 저장
    * 모델의 파라미터만 저장
- 2가지 형태 로딩
    * 전체 저장 모델 파일 ==> 로딩으로 사용 가능
    * 모델 파라미터 저장 파일 ==> 모델 인스턴스 생성 후 층별 파라미터 적용

[1] 모듈 로딩 및 데이터 준비 <hr>

In [ ]:
## [1-1] 모듈 로딩


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from   torch.optim.lr_scheduler import ReduceLROnPlateau
from   torch.utils.data import Dataset, TensorDataset, DataLoader
import EX17_DL.Utils.util_func as uf
import pandas as pd


In [ ]:
## [1-2] 데이터 준비
df = pd.read_csv("./study_score_multi.csv")

X = torch.tensor(df.iloc[:, :-1].values, dtype=torch.float32)
y = torch.tensor(df.iloc[:, -1].values, dtype=torch.float32)

[2] 모델 클래스 정의 <hr>

In [3]:
class Test(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2, 5)
        self.fc2 = nn.Linear(5, 7)
        self.out = nn.Linear(7, 1)

    def forward(self, x):
        out = F.relu(self.fc1(x))
        out = F.relu(self.fc2(out))
        return self.out(out)

[3] 학습 진행 <hr>

In [4]:
## 설정값들
EPOCHS = 10
BS = 2
LR = 0.1

## 저장 모델 파일명
ALL_MODEL = './all_model.pt'    ## 모델 전체 확장자    pt
WEIGHTS_MODEL = './weights'     ## 파라미터 저장 확장자 pth

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


In [5]:
## 인스턴스들
model       = Test().to(DEVICE)
loss_fn     = nn.MSELoss()
optimizer   = optim.Adam(model.parameters(), lr=LR)

dataDS      = TensorDataset(featureTS, targetTS)
dataDL      = DataLoader(dataDS, batch_size=BS, shuffle=True)

validDS     = TensorDataset(featureTS*1.2, targetTS*1.1)
validDL     = DataLoader(dataDS, batch_size=BS)

In [6]:
for x, y in dataDL:
    print(x, y, sep='\n')
    break

tensor([[2., 6.],
        [1., 5.]])
tensor([[8.],
        [6.]])


In [7]:
## 학습진행 ===================================
BEST_LOSS = 100.
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

for epoch in range(EPOCHS):
    #- 학습진행
    train_loss, train_acc = uf.train_one_epoch(model, dataDL, loss_fn, optimizer, DEVICE)

    #- 검증진행
    valid_loss, valid_acc = uf.evaluate(model, validDL, loss_fn, DEVICE)

    #- 모델과 가중치 파일 저장
    if BEST_LOSS > valid_loss:
        #- 모델 전체 저장
        torch.save(model, ALL_MODEL)
        #- 파라미터만 저장
        torch.save(model.state_dict(), f"./{WEIGHTS_MODEL}_{epoch:03}_{valid_loss:.5f}.pth")
        #- 기준 loss 점수 업데이트
        BEST_LOSS = valid_loss
    
    #- 학습상태 출력
    print(f"[LOSS] train:valid={train_loss:.7f}:{valid_loss:.7f} ", end=' ') 
    print(f"[ACC] train:valid={train_acc:.7f}:{valid_acc:.7f} ")


[LOSS] train:valid=70.6426222:11.5720931  [ACC] train:valid=0.0000000:0.0000000 
[LOSS] train:valid=6.6891392:23.5931862  [ACC] train:valid=0.0000000:0.0000000 
[LOSS] train:valid=14.2052649:1.8365623  [ACC] train:valid=0.0000000:0.0000000 
[LOSS] train:valid=4.2936701:10.5175138  [ACC] train:valid=0.0000000:0.0000000 
[LOSS] train:valid=10.1947300:4.5484538  [ACC] train:valid=0.0000000:0.0000000 
[LOSS] train:valid=2.4947201:1.6600281  [ACC] train:valid=0.0000000:0.0000000 
[LOSS] train:valid=3.4522180:5.0255415  [ACC] train:valid=0.0000000:0.0000000 
[LOSS] train:valid=3.4374619:0.3350783  [ACC] train:valid=0.0000000:0.0000000 
[LOSS] train:valid=0.7588538:2.8306532  [ACC] train:valid=0.0000000:0.0000000 
[LOSS] train:valid=3.4525045:1.9622442  [ACC] train:valid=0.0000000:0.0000000 


[4] 모델 파일 사용 <hr>

In [9]:
## [4-1] 가중치 저장 파일 로딩
params = torch.load("./weights_005_1.66003.pth", weights_only=True)

tModel = Test()
tModel.load_state_dict(params)

<All keys matched successfully>

In [10]:
## [4-2] 전체 모델 저장 파일 로딩
allModel = torch.load(ALL_MODEL)
allModel

C:\Users\KDT\AppData\Local\Temp\ipykernel_17244\1771735166.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  allModel = torch.load(ALL_MODEL)


Test(
  (fc1): Linear(in_features=2, out_features=5, bias=True)
  (fc2): Linear(in_features=5, out_features=7, bias=True)
  (out): Linear(in_features=7, out_features=1, bias=True)
)